<a href="https://colab.research.google.com/github/MrFlygerian/NLP-Document-Summary/blob/master/Document_summariser_(experiment).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import  drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!ls "/content/drive/My Drive/Colab Notebooks"
%cd "/content/drive/My Drive/Colab Notebooks"

 control-starting-weight.hdf5		   get_docx_text.py
'Document Summariser (control).ipynb'	   __pycache__
'Document summariser (experiment).ipynb'   README.md
 experiment-starting-weight.hdf5	   wonderland.txt
/content/drive/My Drive/Colab Notebooks


In [19]:
#specific to extracting information from word documents
from get_docx_text import get_docx_text
import os
import sys
#converting letters to numbers
import numpy as np

#deep learning modules
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [20]:
#load files and extract text

#move to the file container folder
%cd '/content/drive/My Drive/Files from retreat/'

files = [f for f in os.listdir('.')]
texts = []
for file in files:
  try:
    texts.append(get_docx_text(file).lower())
  except:
    continue
train_text = ' '.join(texts)

#move back to working directory
%cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Files from retreat
/content/drive/My Drive/Colab Notebooks


In [21]:
#create numberical map using document characters
chars = sorted(list(set(train_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [22]:
# summarize the loaded data
n_chars = len(train_text)
n_vocab = len(chars)

In [23]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = train_text[i:i + seq_length]
	seq_out = train_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
    
n_patterns = len(dataX)

In [24]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [25]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.load_weights('experiment-starting-weight.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
#define checkpoint
filepath="experiment-weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [27]:
#fit model
model.fit(X, y, epochs=30, batch_size=128, callbacks=callbacks_list)

In [28]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


Seed:
" erm goals and projects (to be accomplished by this time next year.)

list your short term goals on t "


In [29]:
# generate characters
for i in range(500):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

he world as a consiitent and some food and the sealization of the chrrch, on the bhristian exists in the forestient of the gorernee to the that hanpens, what he is doonle the country. be able to the president of his pwn success. 

what co you think about your ourpose of achieving the goolowing questions:

what would you like to do the toaie of the gnvernment’s vision of the government’s vision of the government’s vision of the government’s vision of the government’s vision of the government’s vi
Done.
